In [2]:
# Packages
libs = c('dplyr','magrittr','tidyr','readxl','MASS','lubridate','openxlsx','scales','data.table',
         'ggplot2','viridis','gridExtra',
         'loo','StanHeaders','cmdstanr')
for (x in libs) {library(x, character.only = T, warn.conflicts = F)}
set_cmdstan_path('/mnt/d/nlint/Dropbox/Programs/cmdstan')
cmdstan_version()

CmdStan path set to: /mnt/d/nlint/Dropbox/Programs/cmdstan



[1] "2.26.0"

In [3]:
# Load datasets
clusters_info <- readRDS("../data/clusters_info.rds")
clusters_df <- readRDS("../data/clusters_df.rds")
key <- readRDS("../data/key.rds")

In [4]:
# Establish reference date and values
t0 = as.Date("2020-01-01")
daystocalc = 14*3
first_onset_date <- clusters_info %>% pull(firstonset); names(first_onset_date) <- key
last_report_date <- clusters_info %>% pull(lastreport); names(last_report_date) <- key
last_report_day <- as.integer(last_report_date - t0); names(last_report_day) <- key
calc_overall_end_date <- last_report_date + daystocalc - 1; names(calc_overall_end_date) <- key
calc_overall_end_day <- as.integer(calc_overall_end_date - t0); names(calc_overall_end_day) <- key

# Varied parameters
Re = c(0.5, 1.5, 3)
k_mu = c(0.11, 0.25, 0.58)
k_sigma = c(0.051, 0.191, 0.212)
si_mean = c(4.848, 0.610) # mu, sigma
si_par1 = c(2.305, 0.439) # mu, sigma
q = c(0.2, 0.5) # Underascertainment

# Fixed parameters for reporting delay (fit to gamma distribution)
# Code for reporting delay shared elsewhere
gm_mean <- 7.207005
gm_sd <- 4.678002
gm_alpha <- (gm_mean/gm_sd)^2
gm_beta <- gm_mean/(gm_sd^2)
rd_par <- c(gm_alpha, gm_beta)
max_poss_underasc = 50
y = 50

# Fit specs
ncores = parallel::detectCores()
nchains = 4
niter = 2000
nwarm = 1000

## Model without underascertainment

In [5]:
## Model
stan_code <- "functions {
    /* discretized Weibull distribution */
    vector pweibull(real kappa, real theta, int K) {
      vector[K] res;
      for (k in 1:K)
        res[k] = -expm1(-pow(1.0*k/theta, kappa)); // alt to weibull_lcdf
        return append_row(res[1], res[2:K]-res[1:(K-1)]);
    }

    /* discretized gamma distribution */
    vector pgamma(real alpha, real beta, int K) {
      vector[K] res;
      for (k in 1:K)
        res[k] = exp(gamma_lcdf(k | alpha, beta)); // with shape alpha and inverse scale beta
        return append_row(res[1], res[2:K]-res[1:(K-1)]);
    }

    /* vector of convolutions */
    // X: vector of first function, Yrev: reversed vector of the second function
    // N: length of X and Yrev
    // result is vector of length N-1, as the first element equal to zero is omitted
    vector convolution(vector X, vector Yrev, int N) {
        vector[N-1] res;
        res[1] = X[1]*Yrev[N];
        for (k in 2:N-1) // 2:N-1 is equivalent to 2:(N-1)
            res[k] = dot_product(head(X, k), tail(Yrev, k)); // by definition of the convolution
        return res;
    }
    real positive_half_normal_rng(real mu, real sigma) {
      real y = -1;
      while (y < 0)
        y = normal_rng(mu, sigma);
      return y;
    }
}

data {
    int<lower = 1> M; // number of cases reported in the cluster
    int<lower = 0> onset[M]; // vector of onset times

    int<lower = 1> D; // max reporting time
    int<lower = 1> Y; // upper limit for the sum for y, in this case: y = {0, ..., Y-1}

    // reporting delay parameters
       real<lower = 0> rd_par1;
       real<lower = 0> rd_par2;

    // serial interval parameters
       real si_mean_mu;
       real<lower = 0> si_mean_sigma;
       real si_par1_mu;
       real<lower = 0> si_par1_sigma;

    // offspring distribution parameters
       real<lower = 0> R0_par;
       real k_par_mu;
       real<lower = 0> k_par_sigma;
}

generated quantities {
    vector[D] Pr; // probability of extinction

    // offspring distribution parameter k values
       real<lower = 0> k_par = positive_half_normal_rng(k_par_mu, k_par_sigma);

    // serial interval parameters
       real si_mean = normal_rng(si_mean_mu, si_mean_sigma);
       real<lower = 0> si_par1 = normal_rng(si_par1_mu, si_par1_sigma);
       real<lower = 0> si_par2 = si_mean/tgamma(1.0 + 1.0 /si_par1);

    {
       vector[D] ft = pweibull(si_par1, si_par2, D);  // serial interval
       vector[D] ht = pgamma(rd_par1, rd_par2, D);    // reporting delay
       vector[D] htrev;   // reversed reporting delay , required for convolution()
       vector[Y] y;       // vector of indices
       vector[Y] py;      // offspring distribution
       vector[D-1] conv;  // vector of convolutions

        for (t in 1:D)
            htrev[t] = ht[D+1-t];
        conv = cumulative_sum(convolution(ft, htrev, D));

        for (i in 1:Y) {
            y[i] = i - 1;
            py[i] = exp(neg_binomial_2_lpmf(i - 1 | R0_par, k_par)); // mu, phi
        }

        for (t in 1:D) {
            // if the reporting day t is at least two days following the most recent onset date
            // NB: not one day, because of the reporting delay
            if (t <= max(onset) + 1)
                Pr[t] = 0.0;
            else {
                real prodsum = 1.0;
                for (i in 1:M) {
                    int idx = t - onset[i] - 1;
                    real cdf = conv[idx];
                    prodsum *= sum(py .* exp(y * log(cdf)));
                }
                Pr[t] = 1.0 - prodsum;
            }
        }
    }
}"
stan_file = write_stan_file(stan_code)

# Compile model
mod = cmdstan_model(stan_file)

Compiling Stan program...



In [6]:
prnew = list()

(calc_t1 <- Sys.time())

key <- readRDS("../data/key.rds")

for (cl in key) {
    
    R_loop_prnew <- NULL
    
    for (r in seq_along(Re)) {
    
        k_loop_prnew <- NULL

        for (k in seq_along(k_mu)) {
    
            ## Put data in list
            dat <- clusters_df %>% filter(cluster==cl)
            onsets <- dat %>% filter(!is.na(onset)) %>% dplyr::select(onset) %>% pull()
            onsets_imputed <- dat %>% filter(!is.na(onset_imputed)) %>% dplyr::select(onset_imputed) %>% pull()
            Re_par = Re[r]
            k_par_mu = k_mu[k]
            k_par_sigma = k_sigma[k]

            data_list = list(
                onset = as.integer(onsets - t0),
                M = length(onsets),
                D = as.integer(last_report_day[cl] + daystocalc + 1),
                Y = y,

                R0_par = Re_par, 
                k_par_mu = k_par_mu,
                k_par_sigma = k_par_sigma,

                si_mean_mu = si_mean[1],
                si_mean_sigma = si_mean[2],
                si_par1_mu = si_par1[1],
                si_par1_sigma = si_par1[2],

                rd_par1 = rd_par[1],
                rd_par2 = rd_par[2]
            )   


            ## Compile and fit
            fit_reported <- mod$sample(data = data_list,
                                     iter_sampling=niter, 
                                     chains=nchains,
                                     parallel_chains=ncores,
                                     seed=123,
                                     fixed_param=TRUE)

            data_list[["onset"]] <- as.integer(onsets_imputed - t0)
            data_list[["M"]] <- length(onsets_imputed)
            fit_imputed <- mod$sample(data = data_list,
                                     iter_sampling=niter, 
                                     chains=nchains,
                                     parallel_chains=ncores,
                                     seed=123,
                                     fixed_param=TRUE)

            # Save results
            days <- paste0("Pr[",last_report_day[cl]:calc_overall_end_day[cl],"]")
            prnew_reported <- data.frame(fit_reported$summary(c("Pr"), ~quantile(.x, probs = c(0.025, 0.5, 0.975)))) %>%
                              filter(variable %in% days) %>% rename(lci=2, median=3, uci=4) %>% 
                              mutate(Re = Re_par, k = k_par_mu, data_type = "Reported", cluster = cl, 
                                     day = seq(last_report_day[cl], calc_overall_end_day[cl], 1), days_since_last_case = row_number()-1)
            prnew_imputed  <- data.frame(fit_imputed$summary(c("Pr"), ~quantile(.x, probs = c(0.025, 0.5, 0.975)))) %>%
                              filter(variable %in% days) %>% rename(lci=2, median=3, uci=4) %>% 
                              mutate(Re = Re_par, k = k_par_mu, data_type = "Imputed", cluster = cl, 
                                     day = seq(last_report_day[cl], calc_overall_end_day[cl], 1), days_since_last_case = row_number()-1)
            
            k_loop_prnew <- bind_rows(k_loop_prnew, prnew_reported, prnew_imputed)
            
            }

        R_loop_prnew <- bind_rows(R_loop_prnew, k_loop_prnew)
    
      }
    
    prnew[[cl]] <- R_loop_prnew %>% dplyr::select(-variable)
    
}
    
print(Sys.time() - calc_t1)

[1] "2021-03-04 10:09:45 JST"

Running MCMC with 1 chain...

Chain 1 Iteration:    1 / 2000 [  0%]  (Sampling) 
Chain 1 Iteration:  100 / 2000 [  5%]  (Sampling) 
Chain 1 Iteration:  200 / 2000 [ 10%]  (Sampling) 
Chain 1 Iteration:  300 / 2000 [ 15%]  (Sampling) 
Chain 1 Iteration:  400 / 2000 [ 20%]  (Sampling) 
Chain 1 Iteration:  500 / 2000 [ 25%]  (Sampling) 
Chain 1 Iteration:  600 / 2000 [ 30%]  (Sampling) 
Chain 1 Iteration:  700 / 2000 [ 35%]  (Sampling) 
Chain 1 Iteration:  800 / 2000 [ 40%]  (Sampling) 
Chain 1 Iteration:  900 / 2000 [ 45%]  (Sampling) 
Chain 1 Iteration: 1000 / 2000 [ 50%]  (Sampling) 
Chain 1 Iteration: 1100 / 2000 [ 55%]  (Sampling) 
Chain 1 Iteration: 1200 / 2000 [ 60%]  (Sampling) 
Chain 1 Iteration: 1300 / 2000 [ 65%]  (Sampling) 
Chain 1 Iteration: 1400 / 2000 [ 70%]  (Sampling) 
Chain 1 Iteration: 1500 / 2000 [ 75%]  (Sampling) 
Chain 1 Iteration: 1600 / 2000 [ 80%]  (Sampling) 
Chain 1 Iteration: 1700 / 2000 [ 85%]  (Sampling) 
Chain 1 Iteration: 1800 / 2000 [ 90%]  (Sampling) 
C

In [7]:
# Model with underascertainment
stan_code <- "functions {
    /* discretized Weibull distribution */
    vector pweibull(real kappa, real theta, int N) {
      vector[N] res;
      for (k in 1:N)
        res[k] = -expm1(-pow(1.0*k/theta, kappa)); // instead of using Stan function weibull_lcdf it is easier to write it directly
        return append_row(res[1], res[2:N]-res[1:(N-1)]);
    }

    /* discretized gamma distribution */
    vector pgamma(real alpha, real beta, int K) {
      vector[K] res;
      for (k in 1:K)
        res[k] = exp(gamma_lcdf(k | alpha, beta)); // with shape alpha and inverse scale beta
        return append_row(res[1], res[2:K]-res[1:(K-1)]);
    }

    /* vector of convolutions */
    // X: first function, Yrev: reversed version of the second function
    // N: length of X and Yrev
    vector convolution(vector X, vector Yrev, int N) {
        vector[N-1] res;
        res[1] = X[1]*Yrev[N];
        for (k in 2:N-1) // 2:N-1 is equivalent to 2:(N-1)
            res[k] = dot_product(head(X, k), tail(Yrev, k)); // by definition of the convolution
        return append_row(0.0, res);
    }

    real positive_half_normal_rng(real mu, real sigma) {
      real y = -1;
      while (y < 0)
        y = normal_rng(mu, sigma);
      return y;
    }
}

data {
    int<lower = 1> D; // number days in the epicurve
    int day[D]; // vector of days in the epicurve
    int<lower = 0> cases[D]; // number of cases with onsets on that day

    int<lower = 1> Dmax; // maximal reporting time
    int<lower = 1> Y; // upper limit for the sum for y, in this case: y = {0, ..., Y-1}

    int<lower = 1> Umax;

    // reporting delay parameters
    real<lower = 0> rd_par1;
    real<lower = 0> rd_par2;

    // serial interval parameters
    real si_mean_mu;
    real<lower = 0> si_mean_sigma;
    real si_par1_mu;
    real<lower = 0> si_par1_sigma;

    real k_par_mu;
    real<lower = 0> k_par_sigma;

    // offspring distribution parameters
    real<lower = 0> Re_par;

    // underascertainment rate
    real<lower = 0, upper = 1> q;
}

transformed data {
    int max_onset_in_reported_cases;
    for (d in 1:D)
        if (cases[d] > 0)
            max_onset_in_reported_cases = d;
}

parameters {
    simplex[Umax] lambda[D];
}

model {
    vector[Umax] log_lambda[D];
    vector[Umax] lps;
    for (d in 1:D)
        log_lambda[d] = log(lambda[d]);

    for (d in  1:D) {
        lps = log_lambda[d];
        for (U in 1:Umax)
            lps[U] += binomial_lpmf(cases[d] | cases[d] + U - 1, 1.0 - q);
        target += log_sum_exp(lps);
    }
}

generated quantities {
    real Pr[D]; // probability of extinction

    int unreported_cases[D];
    vector[Umax] w[D];
    {
        int comp;
        vector[Umax] lps;
        int M = 0;
        int total_cases[D] = cases;
        for (d in 1:D) {
            lps = log(lambda[d]);
            for (U in 1:Umax)
                lps[U] += binomial_lpmf(cases[d] | cases[d] + U - 1, 1.0 - q);
            w[d] = exp(lps - log_sum_exp(lps));
            comp = categorical_rng(w[d]);
            // if we assume that the underascertained cases can be only within the generation time interval of the latest case
            total_cases[d] += (d <= max_onset_in_reported_cases + 5) ? comp - 1 : 0;
            // otherwise
            // total_cases[d] += comp - 1;
            M += total_cases[d];
        }

        // for our procedure below we need to have a vector of onset times not vector of incidence per day
        int onset[M];
        int kk = 0;
        for (d in 1:D)
            for (case_ in 1:total_cases[d]) {
                kk += 1;
                onset[kk] = d;
            }
        int max_onset = max(onset);

        // offspring distribution parameter k values
           real k_par = positive_half_normal_rng(k_par_mu, k_par_sigma);

        // serial interval parameters
           real si_mean = positive_half_normal_rng(si_mean_mu, si_mean_sigma);
           real si_par1 = positive_half_normal_rng(si_par1_mu, si_par1_sigma);
           real si_par2 = si_mean / tgamma(1.0 + 1.0 / si_par1);

        vector[D] ft = pweibull(si_par1, si_par2, D); // serial interval
        vector[D] ht = pgamma(rd_par1, rd_par2, D); // reporting delay
        vector[D] htrev; // reversed reporting delay, required for convolution()
        for (d in 1:D)
            htrev[d] = ht[D + 1 - d];
        vector[D] conv = cumulative_sum(convolution(ft, htrev, D)); // vector of convolutions

        vector[Y] y; // vector of indices
        vector[Y] log_py; // offspring distribution
        for (k in 1:Y) {
            y[k] = k - 1;
            log_py[k] = neg_binomial_2_lpmf(k - 1 | Re_par, k_par);
        }

        for (d in 1:D) {
            if (d <= max_onset + 1)
                Pr[d] = 1.0;
            else {
                real log_prodsum = 0.0;
                int idx; real cdf;
                for (i in 1:M) {
                    idx = d - onset[i];
                    cdf = conv[idx];
                    log_prodsum += log_sum_exp(log_py + y * log(cdf));
                }
                Pr[d] = 1.0 - exp(log_prodsum);
            }
        }
    }
}"
stan_file_with_ut = write_stan_file(stan_code)

# Compile model
mod_with_ut = cmdstan_model(stan_file_with_ut)

Compiling Stan program...



In [8]:
# Model with underascertainment

prnew_with_ut = list()

(calc_t1 <- Sys.time())

key <- readRDS("../data/key.rds")

for (cl in key) {
    
    df <- clusters_df %>% filter(cluster==cl) %>% count(onset_imputed) %>% rename(date=onset_imputed, n_imp=n) %>%                          ## imputed onsets by day
          left_join(clusters_df %>% filter(cluster==cl) %>% count(onset), by=c("date"="onset")) %>%                                         ## reported onsets by day 
          complete(date = seq.Date(first_onset_date[cl], last_report_date[cl]+daystocalc, by="day")) %>% replace_na(list(n=0, n_imp=0)) %>%  ## add 0 case days
          mutate(day = seq(0, as.numeric((last_report_date[cl]+daystocalc)-first_onset_date[cl]), 1)) %>%                                    ## add day d to first onset
          rbind(data.frame(date=seq.Date(first_onset_date[cl]-5,first_onset_date[cl]-1, 1), n_imp=0, n=0, day=seq(-5,-1,1))) %>%            ## add 5 day initial buffer
          arrange(date)
    
    q_loop_prnew <- NULL
    
    for (i in seq_along(q)) {
    
        R_loop_prnew <- NULL

        for (r in seq_along(Re)) {

            k_loop_prnew <- NULL

            for (k in seq_along(k_mu)) {

                Re_par = Re[r]
                k_par_mu = k_mu[k]
                k_par_sigma = k_sigma[k]


                data_list = list(
                    D = nrow(df),
                    Dmax = max(df$day)+1,
                    day = df$day,
                    cases = df$n_imp,

                    Y = 100,
                    Umax = max_poss_underasc,
                    q = q[i],

                    Re_par = Re_par, 
                    k_par_mu = k_par_mu,
                    k_par_sigma = k_par_sigma,

                    si_mean_mu = si_mean[1],
                    si_mean_sigma = si_mean[2],
                    si_par1_mu = si_par1[1],
                    si_par1_sigma = si_par1[2],

                    rd_par1 = rd_par[1],
                    rd_par2 = rd_par[2]
                )

                ## Fit model
                fit <- mod_with_ut$sample(data = data_list, 
                                     iter_sampling=niter, 
                                     iter_warmup=nwarm, 
                                     chains=nchains,
                                     parallel_chains=ncores,
                                     save_warmup = TRUE,
                                     seed=123)

                # Save results
                days_lower <- df %>% filter(date==last_report_date[cl]) %>% pull(day)
                max_calc <- nrow(fit$summary(c("Pr")))
                max_poss <- days_lower+daystocalc+1
                days_range <- seq(days_lower+1, ifelse(max_calc<max_poss, max_calc, max_poss), 1)
                days <- paste0("Pr[",days_range,"]")
                prnew_ut <- data.frame(fit$summary(c("Pr"), ~quantile(.x, probs = c(0.025, 0.5, 0.975)))) %>%
                               filter(variable %in% days) %>% rename(lci=2, median=3, uci=4) %>% 
                               mutate(Re = Re_par, k = k_par_mu, data_type = sprintf("%i%% underascertainment", q[i]*100), cluster = cl, 
                                      day = days_range, days_since_last_case = row_number()-1) %>% dplyr::select(-variable)

                k_loop_prnew <- bind_rows(k_loop_prnew, prnew_ut)

                }

            R_loop_prnew <- bind_rows(R_loop_prnew, k_loop_prnew)

            }

        q_loop_prnew <- bind_rows(q_loop_prnew, R_loop_prnew)

        }
    
    prnew_with_ut[[cl]] <- q_loop_prnew
    
}
    
print(Sys.time() - calc_t1)

[1] "2021-03-04 10:13:48 JST"

Running MCMC with 4 chains, at most 48 in parallel...

Chain 1 Iteration:    1 / 3000 [  0%]  (Warmup) 
Chain 2 Iteration:    1 / 3000 [  0%]  (Warmup) 
Chain 3 Iteration:    1 / 3000 [  0%]  (Warmup) 
Chain 4 Iteration:    1 / 3000 [  0%]  (Warmup) 
Chain 3 Iteration:  100 / 3000 [  3%]  (Warmup) 
Chain 1 Iteration:  100 / 3000 [  3%]  (Warmup) 
Chain 2 Iteration:  100 / 3000 [  3%]  (Warmup) 
Chain 4 Iteration:  100 / 3000 [  3%]  (Warmup) 
Chain 2 Iteration:  200 / 3000 [  6%]  (Warmup) 
Chain 3 Iteration:  200 / 3000 [  6%]  (Warmup) 
Chain 4 Iteration:  200 / 3000 [  6%]  (Warmup) 
Chain 1 Iteration:  200 / 3000 [  6%]  (Warmup) 
Chain 3 Iteration:  300 / 3000 [ 10%]  (Warmup) 
Chain 2 Iteration:  300 / 3000 [ 10%]  (Warmup) 
Chain 4 Iteration:  300 / 3000 [ 10%]  (Warmup) 
Chain 1 Iteration:  300 / 3000 [ 10%]  (Warmup) 
Chain 2 Iteration:  400 / 3000 [ 13%]  (Warmup) 
Chain 3 Iteration:  400 / 3000 [ 13%]  (Warmup) 
Chain 4 Iteration:  400 / 3000 [ 13%]  (Warmup) 
Chain 1 Iterat

In [9]:
all_prnew <- bind_rows(rbindlist(prnew), rbindlist(prnew_with_ut))

In [10]:
saveRDS(all_prnew, "../results/prnew.rds")